# Desnormalización de coordenadas

Este notebook demuestra cómo es posible desnormalizar los datos obtenidos en las predicciones realizadas por yolo.

Trsas realizar las predicciones, los resutlados quedan guardadso en runs/detect/predict/labels.

Estos txt contienen tanto la etiqueta predicha como la imagen original. Para poder seguir usando estas coordenada, será necesario desnormalizarlas.

In [2]:
import utils.yolo_fun as yolo_fun
from utils.reverse_pixel import *
import utils.img_fun as img_fun
import os
import pandas as pd
from tqdm import tqdm  
import rasterio
from rasterio.windows import Window
from rasterio.errors import RasterioIOError
import numpy as np
import shutil
from sklearn.model_selection import train_test_split


# Indicamos el input path de la imagen que queremos desnormalizar
input_img = './datasets/penguin_dataset/images/train/recorte_6_-59.21061762925279_-62.28551585752274.tiff'



# Indicamos el path con el txt de la label asociada
coords_csv = './datasets/penguin_dataset/labels/train/recorte_6_-59.21061762925279_-62.28551585752274.txt'


# Finalmente indicamos donde queremos guardar el csv con las coordenadas desnormalizadas
output_dir = 'coords/ejemplo_reverse.csv'


df_coords_normalized = pd.read_csv(coords_csv, sep=' ', header=None)


# Ejemplo de uso

A continuación vemos como la denormalización recibe los tres parámetros.

In [2]:
img_dir = './runs/detect/predict/recorte_6_-59.21074959059679_-62.28591513099474.jpg'
label_dir = './runs/detect/predict/labels/recorte_6_-59.21074959059679_-62.28591513099474.txt'
df_coords_normalized = pd.read_csv(label_dir, sep=' ', header=None)


denormalize_yolo_coords(
    tiff_file = input_img,
    coords_normalizadas = df_coords_normalized,
    output_file = output_dir
)

### Uso dentro de un bucle

Idealmente, utilizaremos la función dentro de un bucle que recorra todos los archivos de la carpeta. El nombre de cada archivo será exactamente el mismo que el de la imagen al que se refieren todas las coordenadas. 


In [8]:

#! ASEGURARSE DE QUE LAS PREDICCIONES Y LAS IMÁGENES USAN UN SISTEMA DE COORDENADSA VÁLIDO

# Aquí estarían las imágenes usadas en la predicción
img_dir = './runs/detect/predict/'

# Directorio con las rutas en txt
label_dir = './runs/detect/predict/labels'

# Output path desnormalizado
output_dir = './runs/detect/predict/labels_denormalized/'


for file in os.listdir(label_dir):
    df_coords_normalized = pd.DataFrame()
    print("Limpio", df_coords_normalized.head())
    if file.endswith('.txt'):
        file_path = os.path.join(label_dir, file)
        tiff_file = os.path.join(img_dir, file.replace('.txt', '.jpg'))


        df_coords_normalized = pd.read_csv(file_path, sep=' ', header=None)
        print(f"\nProcesando archivo: {file}")
        print(f"Ruta del archivo TIFF: {tiff_file}")
        print(df_coords_normalized.head())
       
        # Llamar a la función con datos frescos
        denormalize_yolo_coords(
            tiff_file=tiff_file,
            coords_normalizadas=df_coords_normalized.copy(), 
            output_file=os.path.join(output_dir, file)
        )



Limpio Empty DataFrame
Columns: []
Index: []

Procesando archivo: recorte_6_-59.21074959059679_-62.285607997554735.txt
Ruta del archivo TIFF: ./runs/detect/predict/recorte_6_-59.21074959059679_-62.285607997554735.jpg
   0         1         2         3         4
0  0  0.119377  0.860472  0.047167  0.046480
1  0  0.154885  0.830721  0.050240  0.048833
2  0  0.088193  0.642265  0.051782  0.051671
3  0  0.072185  0.802036  0.049408  0.048735
4  0  0.698810  0.916793  0.043734  0.044084
Limpio Empty DataFrame
Columns: []
Index: []

Procesando archivo: recorte_6_-59.21074959059679_-62.285761564274736.txt
Ruta del archivo TIFF: ./runs/detect/predict/recorte_6_-59.21074959059679_-62.285761564274736.jpg
   0         1         2         3         4
0  0  0.736862  0.071425  0.039890  0.039907
1  0  0.758949  0.043315  0.039722  0.042284
2  0  0.681362  0.081937  0.041130  0.041101
3  0  0.771675  0.110211  0.043051  0.041048
4  0  0.703376  0.019499  0.046798  0.038997
Limpio Empty DataFrame
Col

c:\Users\Administrador.CRISASUSESTUDIO\AppData\Local\Programs\Python\Python312\Lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
c:\Users\Administrador.CRISASUSESTUDIO\AppData\Local\Programs\Python\Python312\Lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
c:\Users\Administrador.CRISASUSESTUDIO\AppData\Local\Programs\Python\Python312\Lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
c:\Users\Administrador.CRISASUSESTUDIO\AppData\Local\Programs\Python\Python312\Lib\site-

Limpio Empty DataFrame
Columns: []
Index: []

Procesando archivo: recorte_7_-59.21011298020679_-62.28606869771474.txt
Ruta del archivo TIFF: ./runs/detect/predict/recorte_7_-59.21011298020679_-62.28606869771474.jpg
   0         1         2         3         4
0  0  0.296100  0.666784  0.044613  0.045387
1  0  0.156942  0.860680  0.045651  0.045117
2  0  0.318089  0.453968  0.046900  0.046403
3  0  0.366076  0.443003  0.045125  0.045595
4  0  0.121723  0.335469  0.046113  0.046344
Limpio Empty DataFrame
Columns: []
Index: []

Procesando archivo: recorte_7_-59.21011298020679_-62.286222264434734.txt
Ruta del archivo TIFF: ./runs/detect/predict/recorte_7_-59.21011298020679_-62.286222264434734.jpg
   0         1         2         3         4
0  0  0.193758  0.071559  0.037819  0.037702
1  0  0.170905  0.027088  0.039259  0.037820


c:\Users\Administrador.CRISASUSESTUDIO\AppData\Local\Programs\Python\Python312\Lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
c:\Users\Administrador.CRISASUSESTUDIO\AppData\Local\Programs\Python\Python312\Lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
c:\Users\Administrador.CRISASUSESTUDIO\AppData\Local\Programs\Python\Python312\Lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


### Merge de las coordenas resultantes

Como último paso, y para seguir utilizando dichas coordenas en qgis o rasterio, bastaría con funcionarlas todas en el mismo documento.

In [12]:
output_dir = './runs/detect/predict/merged_coords.csv'
denormalized_dir = './runs/detect/predict/labels_denormalized/'

# Mergeamos todos los csv en uno solo
merged = pd.DataFrame()
for file in os.listdir(denormalized_dir):
    df = pd.read_csv(os.path.join(denormalized_dir, file), sep=' ', header=None)
    df = df.iloc[:, [1, 2]]
    merged = pd.concat([merged, df])

merged.to_csv(output_dir, index=False)
   

In [9]:
# Aquí estarían las imágenes usadas en la predicción
img_dir = './runs/detect/predict/'

# Directorio con las rutas en txt
label_dir = './runs/detect/predict/labels'

# Output path desnormalizado
output_dir = './runs/detect/predict/labels_denormalized/'

df = pd.read_csv('./runs/detect/predict/labels/recorte_6_-59.21074959059679_-62.28591513099474.txt', header=None, sep=' ')
df.head()

,0,1,2,3,4
0,0,0.908275,0.947678,0.045063,0.045025
1,0,0.781967,0.808611,0.046154,0.046348
2,0,0.909873,0.706560,0.044804,0.045108
3,0,0.793620,0.851954,0.048031,0.046318
4,0,0.878297,0.816401,0.043309,0.044051
